# Manhattan Neighborhood Analysis

Need to figure out which neighborhoods I would want to move to in Manhattan based on venues located throughout the city.

### Import Libraries

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

### Open the File and Load the Data

In [3]:
with open('nyu-2451-34572-geojson.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
neighborhood_data = newyork_data['features']
neighborhood_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

In [5]:
column_names = ['Borough','Neighborhood','Latitude','Longitude']

neighborhoods = pd.DataFrame(columns = column_names)

In [6]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [7]:
for data in neighborhood_data:
    borough = data['properties']['borough']
    neighborhood_name = data['properties']['name']
    
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon},
                                          ignore_index=True)

In [8]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [9]:
len(neighborhoods['Borough'].unique())

5

In [10]:
neighborhoods.shape

(306, 4)

In [12]:
address = 'New York City, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


In [13]:
latitude

40.7308619

In [14]:
longitude

-73.9871558

In [15]:
map_newyork = folium.Map(location=[latitude,longitude], zoom_start=10)

for lat,lng,borough,neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_newyork)
    
map_newyork

### Set Up the Connection to Foursquare API and Get Venue Data

In [16]:
CLIENT_ID = 'UFZOZC1HROQY24HZCZGX1XP041DZORGU510DDLUV4MS1JYLQ' # your Foursquare ID
CLIENT_SECRET = 'ZEUWPPVPNBFXZFVGWRYLFBVN43TT1FXBBSIZX3EKXSDNNWD0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UFZOZC1HROQY24HZCZGX1XP041DZORGU510DDLUV4MS1JYLQ
CLIENT_SECRET:ZEUWPPVPNBFXZFVGWRYLFBVN43TT1FXBBSIZX3EKXSDNNWD0


In [17]:
manhattan_data = neighborhoods[neighborhoods['Borough']=='Manhattan'].reset_index(drop=True)

In [18]:
neighborhood_latitude = manhattan_data.loc[0,'Latitude']
neighborhood_longitude = manhattan_data.loc[0,'Longitude']
radius = 500
LIMIT = 100
latitude = neighborhood_latitude
longitude = neighborhood_longitude
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=UFZOZC1HROQY24HZCZGX1XP041DZORGU510DDLUV4MS1JYLQ&client_secret=ZEUWPPVPNBFXZFVGWRYLFBVN43TT1FXBBSIZX3EKXSDNNWD0&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=500&limit=100'

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c7322d24c1f6762f70c6e0b'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b4429abf964a52037f225e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1ca941735',
         'name': 'Pizza Place',
         'pluralName': 'Pizza Places',
         'primary': True,
         'shortName': 'Pizza'}],
       'delivery': {'id': '72548',
        'provider': {'icon': {'name': '/delivery_provider_seamless_20180129.png',
          'prefix': 'https://fastly.4sqi.net/img/general/cap/',
          'sizes': [40, 50]},
         'name': 'seamless'},
        'url': 'https://www.seamless.com/menu/arturos-pizza-5189-broadway-ave-new-york/72548?affiliate=1131&utm_source=foursquare-affiliat

In [22]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues = nearby_venues.loc[:,filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = list(col.split(".")[-1] for col in nearby_venues.columns)
nearby_venues.head()

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Starbucks,Coffee Shop,40.877531,-73.905582
4,Land & Sea Restaurant,Seafood Restaurant,40.877885,-73.905873


### Explore Neighborhoods in Manhattan

In [24]:
def getNearbyVenues (names, latitudes, longitudes, radius=500):
    
    venues_list = []
    
    for name,lat,lng in zip(names, latitudes, longitudes):
        print(name)
    
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                LIMIT)

        results = requests.get(url).json()["response"]['groups'][0]['items']

        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues = pd.DataFrame(list(item for venue_list in venues_list for item in venue_list))
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
    

In [25]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude'])

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [82]:
manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant


In [83]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,100,100,100,100,100,100
Carnegie Hill,100,100,100,100,100,100
Central Harlem,46,46,46,46,46,46
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,44,44,44,44,44,44
East Village,100,100,100,100,100,100


In [84]:
len(manhattan_venues['Venue Category'].unique())

331

### Analyze Each Neighborhood

In [85]:
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="",prefix_sep="")

In [86]:
manhattan_onehot.shape

(3309, 331)

In [87]:
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood']
manhattan_onehot.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,

In [88]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Sob

In [89]:
num_top_venues = 3

for hood in manhattan_grouped['Neighborhood']:
    print("-----"+hood+"-----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood']== hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

-----Battery Park City-----
         venue  freq
0  Coffee Shop  0.08
1         Park  0.07
2        Hotel  0.05


-----Carnegie Hill-----
            venue  freq
0     Pizza Place  0.06
1     Coffee Shop  0.05
2  Cosmetics Shop  0.05


-----Central Harlem-----
                venue  freq
0  African Restaurant  0.07
1  Chinese Restaurant  0.07
2  Seafood Restaurant  0.04


-----Chelsea-----
                venue  freq
0         Coffee Shop  0.07
1  Italian Restaurant  0.06
2      Ice Cream Shop  0.05


-----Chinatown-----
                   venue  freq
0     Chinese Restaurant  0.09
1        Bubble Tea Shop  0.06
2  Vietnamese Restaurant  0.04


-----Civic Center-----
                  venue  freq
0  Gym / Fitness Center  0.06
1                Bakery  0.05
2    Italian Restaurant  0.05


-----Clinton-----
                  venue  freq
0               Theater  0.13
1  Gym / Fitness Center  0.04
2           Coffee Shop  0.04


-----East Harlem-----
                venue  freq
0  Mexican R

### Putting this in a Pandas Dataframe

In [90]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [91]:
num_top_venues = 10 # must be less than 20

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']


In [92]:
for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind,:],num_top_venues)
    
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Coffee Shop,Park,Hotel,Wine Shop,Italian Restaurant,Department Store,Clothing Store,Burger Joint,Memorial Site,Fountain
1,Carnegie Hill,Pizza Place,Cosmetics Shop,Coffee Shop,Café,French Restaurant,Spa,Bookstore,Bar,Gym,Japanese Restaurant
2,Central Harlem,Chinese Restaurant,African Restaurant,Fried Chicken Joint,Gym / Fitness Center,Seafood Restaurant,French Restaurant,Cosmetics Shop,American Restaurant,Park,Beer Bar
3,Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,Nightclub,Bakery,American Restaurant,Theater,Seafood Restaurant,Hotel,Market
4,Chinatown,Chinese Restaurant,Bubble Tea Shop,Dim Sum Restaurant,American Restaurant,Vietnamese Restaurant,Cocktail Bar,Salon / Barbershop,Bakery,Hotpot Restaurant,Noodle House


### Run Clustering Analysis to Figure Out Which Neighborhoods to Live In

In [93]:
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

In [94]:
kmeans.labels_

array([2, 2, 0, 2, 0, 2, 2, 3, 0, 2, 2, 2, 0, 3, 2, 3, 2, 2, 0, 0, 3, 0,
       2, 2, 1, 2, 2, 2, 3, 2, 4, 2, 2, 3, 2, 2, 0, 3, 0, 2], dtype=int32)

In [95]:
clust_centers = pd.DataFrame(kmeans.cluster_centers_, columns=list(manhattan_grouped_clustering.columns))

In [96]:
manhattan_merged = manhattan_data
manhattan_merged['Cluster'] = kmeans.labels_
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#### Cluster 0

In [97]:
manhattan_merged.loc[manhattan_merged['Cluster']==0, manhattan_merged.columns[[1]+ list(range(5,manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Café,Bakery,Mobile Phone Shop,Caribbean Restaurant,Chinese Restaurant,Sandwich Place,Shoe Store,Tapas Restaurant,Grocery Store,Gym
4,Hamilton Heights,Mexican Restaurant,Deli / Bodega,Coffee Shop,Café,Pizza Place,Liquor Store,Park,Cocktail Bar,Sandwich Place,School
8,Upper East Side,Italian Restaurant,Exhibit,Art Gallery,Juice Bar,Bakery,Coffee Shop,Gym / Fitness Center,Boutique,French Restaurant,Hotel
12,Upper West Side,Italian Restaurant,Bar,Vegetarian / Vegan Restaurant,Burger Joint,Indian Restaurant,Bakery,Coffee Shop,Wine Bar,Sushi Restaurant,Mediterranean Restaurant
18,Greenwich Village,Italian Restaurant,Sushi Restaurant,French Restaurant,Clothing Store,Indian Restaurant,Boutique,Café,Seafood Restaurant,Chinese Restaurant,Caribbean Restaurant
19,East Village,Ice Cream Shop,Bar,Wine Bar,Cocktail Bar,Mexican Restaurant,Pizza Place,Ramen Restaurant,Chinese Restaurant,Record Shop,Café
21,Tribeca,Italian Restaurant,Park,American Restaurant,Spa,Café,Boutique,Wine Bar,Coffee Shop,Gym,Greek Restaurant
36,Tudor City,Mexican Restaurant,Park,Pizza Place,Café,Greek Restaurant,Deli / Bodega,Hotel,Diner,Dog Run,Asian Restaurant
38,Flatiron,Italian Restaurant,Yoga Studio,Gym / Fitness Center,Gym,American Restaurant,Women's Store,New American Restaurant,Dessert Shop,Bakery,Cycle Studio


In [98]:
clust_centers.iloc[0,:].sort_values(ascending=False).head(10)

Italian Restaurant     0.041686
Chinese Restaurant     0.039643
American Restaurant    0.024918
Bakery                 0.023643
Seafood Restaurant     0.022473
Coffee Shop            0.021508
Café                   0.020102
Ice Cream Shop         0.019599
Bar                    0.019570
French Restaurant      0.018875
Name: 0, dtype: float64

#### Cluster 1

In [99]:
manhattan_merged.loc[manhattan_merged['Cluster']==1, manhattan_merged.columns[[1]+ list(range(5,manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,West Village,Italian Restaurant,New American Restaurant,Cosmetics Shop,Gastropub,Jazz Club,Wine Bar,Bakery,French Restaurant,Park,American Restaurant


In [100]:
clust_centers.iloc[1,:].sort_values(ascending=False).head(10)

Korean Restaurant       0.17
Japanese Restaurant     0.04
Hotel                   0.04
Hotel Bar               0.04
Coffee Shop             0.04
Cosmetics Shop          0.04
Bakery                  0.03
Cocktail Bar            0.03
Italian Restaurant      0.03
Gym / Fitness Center    0.03
Name: 1, dtype: float64

#### Cluster 2

In [101]:
manhattan_merged.loc[manhattan_merged['Cluster']==2, manhattan_merged.columns[[1]+ list(range(5,manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Discount Store,Coffee Shop,Yoga Studio,Deli / Bodega,Supplement Shop,Steakhouse,Shopping Mall,Shoe Store,Seafood Restaurant,Sandwich Place
1,Chinatown,Chinese Restaurant,Bubble Tea Shop,Dim Sum Restaurant,American Restaurant,Vietnamese Restaurant,Cocktail Bar,Salon / Barbershop,Bakery,Hotpot Restaurant,Noodle House
3,Inwood,Café,Mexican Restaurant,Pizza Place,Lounge,Deli / Bodega,Wine Bar,Bakery,American Restaurant,Park,Restaurant
5,Manhattanville,Deli / Bodega,Italian Restaurant,Mexican Restaurant,Chinese Restaurant,Seafood Restaurant,Japanese Curry Restaurant,Sushi Restaurant,Supermarket,Burger Joint,Spanish Restaurant
6,Central Harlem,Chinese Restaurant,African Restaurant,Fried Chicken Joint,Gym / Fitness Center,Seafood Restaurant,French Restaurant,Cosmetics Shop,American Restaurant,Park,Beer Bar
9,Yorkville,Italian Restaurant,Coffee Shop,Bar,Gym,Pizza Place,Deli / Bodega,Japanese Restaurant,Mexican Restaurant,Sushi Restaurant,Wine Shop
10,Lenox Hill,Italian Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Gym / Fitness Center,Sporting Goods Shop,Burger Joint,Gym,Turkish Restaurant,Art Gallery
11,Roosevelt Island,Park,Sandwich Place,Rental Car Location,Bus Station,Bubble Tea Shop,Bridge,Supermarket,Farmers Market,Metro Station,Outdoors & Recreation
14,Clinton,Theater,Coffee Shop,American Restaurant,Italian Restaurant,Gym / Fitness Center,Gym,Hotel,Spa,Wine Shop,New American Restaurant
16,Murray Hill,Coffee Shop,Hotel,French Restaurant,Italian Restaurant,Japanese Restaurant,Sandwich Place,Bar,Salon / Barbershop,Gym,Spa


In [102]:
clust_centers.iloc[2,:].sort_values(ascending=False).head(10)

Coffee Shop             0.050445
Italian Restaurant      0.044553
American Restaurant     0.028367
Hotel                   0.023756
Gym / Fitness Center    0.023301
Gym                     0.023034
Park                    0.020221
Pizza Place             0.018860
Bakery                  0.018636
Café                    0.017995
Name: 2, dtype: float64

#### Cluster 3

In [103]:
manhattan_merged.loc[manhattan_merged['Cluster']==3, manhattan_merged.columns[[1]+ list(range(5,manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,East Harlem,Mexican Restaurant,Bakery,Deli / Bodega,Latin American Restaurant,Thai Restaurant,French Restaurant,Seafood Restaurant,Taco Place,Street Art,Steakhouse
13,Lincoln Square,Gym / Fitness Center,Theater,Concert Hall,Plaza,Italian Restaurant,Café,French Restaurant,Opera House,Indie Movie Theater,Performing Arts Venue
15,Midtown,Hotel,Clothing Store,Theater,Coffee Shop,Steakhouse,Bookstore,Bakery,Sporting Goods Shop,Spa,Cocktail Bar
20,Lower East Side,Café,Coffee Shop,Ramen Restaurant,Bakery,Pizza Place,Japanese Restaurant,Shoe Store,Chinese Restaurant,Latin American Restaurant,Sandwich Place
28,Battery Park City,Coffee Shop,Park,Hotel,Wine Shop,Italian Restaurant,Department Store,Clothing Store,Burger Joint,Memorial Site,Fountain
33,Midtown South,Korean Restaurant,Japanese Restaurant,Hotel,Hotel Bar,Cosmetics Shop,Coffee Shop,Cocktail Bar,Gym / Fitness Center,Bakery,Italian Restaurant
37,Stuyvesant Town,Bar,Boat or Ferry,Park,Playground,Gas Station,Pet Service,Farmers Market,Basketball Court,Baseball Field,Cocktail Bar


In [104]:
clust_centers.iloc[3,:].sort_values(ascending=False).head(10)

Mexican Restaurant           0.054732
Deli / Bodega                0.045361
Café                         0.042605
Pizza Place                  0.040497
Park                         0.038311
Coffee Shop                  0.030977
Bakery                       0.029293
Sandwich Place               0.025804
Latin American Restaurant    0.021959
Liquor Store                 0.017513
Name: 3, dtype: float64

#### Cluster 4

In [105]:
manhattan_merged.loc[manhattan_merged['Cluster']==4, manhattan_merged.columns[[1]+ list(range(5,manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Carnegie Hill,Pizza Place,Cosmetics Shop,Coffee Shop,Café,French Restaurant,Spa,Bookstore,Bar,Gym,Japanese Restaurant


In [106]:
clust_centers.iloc[4,:].sort_values(ascending=False).head(10)

Bar                 0.210526
Boat or Ferry       0.105263
Park                0.105263
Playground          0.105263
Gas Station         0.052632
Pet Service         0.052632
Farmers Market      0.052632
Basketball Court    0.052632
Baseball Field      0.052632
Cocktail Bar        0.052632
Name: 4, dtype: float64

### Attach Cluster Labels to Find Places that Influence the Cluster Analysis the Most

In [110]:
manhattan_grouped['Cluster'] = kmeans.labels_
manhattan_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Sob

In [111]:
manhattan_grouped.shape

X = manhattan_grouped.iloc[:,1:-1].values
y = manhattan_grouped.iloc[:,-1].values
manhattan_grouped.columns[1:-1]

Index(['Accessories Store', 'Adult Boutique', 'Afghan Restaurant',
       'African Restaurant', 'American Restaurant', 'Animal Shelter',
       'Antique Shop', 'Arcade', 'Arepa Restaurant', 'Argentinian Restaurant',
       ...
       'Volleyball Court', 'Watch Shop', 'Waterfront', 'Weight Loss Center',
       'Whisky Bar', 'Wine Bar', 'Wine Shop', 'Wings Joint', 'Women's Store',
       'Yoga Studio'],
      dtype='object', length=331)

In [112]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(max_depth=100, n_jobs=-1,n_estimators=1000)

In [113]:
rf.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=100, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [114]:
df_test = pd.DataFrame({'feature': manhattan_grouped.columns[1:-1], 'feature_import' : rf.feature_importances_})

In [117]:
df_test.sort_values('feature_import', axis=0, ascending=False).head(10)

,feature,feature_import
62,Chinese Restaurant,0.050541
175,Latin American Restaurant,0.030779
191,Mexican Restaurant,0.023075
180,Liquor Store,0.021251
164,Italian Restaurant,0.020983
87,Deli / Bodega,0.020854
284,Steakhouse,0.019614
68,Coffee Shop,0.019327
58,Caribbean Restaurant,0.018861
228,Pet Store,0.018237


It can be seen that the above venue types are the most influential when it comes to the cluster analysis.

### Conclusion

Based on the above cluster analysis, I would be looking for a place in a neighborhood that is contained in cluster 0 because I like the combination of American, Chinese, Italian, and Seafood restaurants. Therefore, I would limit my housing search to the following neighborhoods:

* Washington Heights
* Hamilton Heights
* Upper East Side
* Upper West Side
* Greenwich Village
* East Village
* Tribeca
* Tudor City
* Flatiro